#                                          Timeseries Powerflow

The time series module is designed for the simulation of time based operations and is linked to the control module. Within a time series simulation controllers are used to update values of different elements in each time step in a loop. Refer to time series overview for details and to example for an easy example.

In pandapower, the timeseries module have the following steps.

1. Load or create your pandapower network
2. Create the dataframe containing the timeseries profile
3. Create the controllers to update the values of the variables you want (Normally P and Q)
4. Define which variables do you want to have as an output
5. Run the main timeseries function and plot results

First, lets import all the functions we need:

In [ ]:
import os
import numpy as np
import pandas as pd
import tempfile
import random
import pandapower as pp
import plotly.express as px
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl

## Step 1 - Load your pandapower network

In [ ]:
# You can use to_excel and from_excel functions to export and import pandapower networks as excel sheets

net = ?

print(net)

## Step 2 - Creating the datasource profiles

As a start, we are going to use the same profiles of the Exercise 2:

<img style="float: center;" src="Figures/profiles.png" width="80%">                               

In [ ]:
profiles = pd.DataFrame(index = [1,2,3,4,5,6], columns = ['load1_p', 'gen1_p'])

profiles['load1_p'] = [0.66, 0.95, 0.82, 0.94, 0.99, 0.68]

profiles['gen1_p'] = [0.15, 0.65, 0.94, 0.92, 0.62, 0.21]

print(profiles)

Remember that the controllers set the controlled variables to the exact value that is given by the datasource. Therefore we must provide the profiles in the units used by pandapower, power must be given in MW and MVAR, voltages in p.u. and currents in kA.

In [ ]:
# We have to multiply the p.u. values to the rated power of the load or generator we are controlling

profiles['load1_p'] = profiles['load1_p']*net.load.p_mw[0]

profiles['gen1_p'] = profiles['gen1_p']*net.gen.p_mw[0]

print(profiles)

Now we can create our pandapower datasource using our dataframe

In [ ]:
ds = DFData(profiles)

## Step 3 - Creating the controllers

### To create a controller we need the following inputs:

**element**: which type of pandapower element we are controlling as a string (Ex: 'load', 'sgen', 'line').

**variable**: what aspect of this element we want to control also as string (Ex: 'p_mw', 'q_mvar', 'vm_pu').

**element_index**: specify what is the index of the element we are controlling (integer).

**data_source**: the datasource module containing the dataframe with profile data.

**profile_name**: what column of the profiles dataframe do you want to assign to this controller. To make it easier, usually we name the profiles with the index of the element we want to control (Ex: 'load1_p', 'sgen7_q').


In [ ]:
# We need to define the element, the variable and also the index of what we want to control.

ConstControl(net, element='load', variable='p_mw', element_index=[0], data_source=ds, profile_name=["load1_p"])

ConstControl(net, element='gen', variable='p_mw', element_index=[0], data_source=ds, profile_name=["gen1_p"])

# Step 4 - Selecting Output Variables

To export the results of the timeseries powerflow pandapower uses a module called "OutputWriter". 

In [ ]:
os.getcwd()

In [ ]:
time_steps = range(1, profiles.shape[0]+1)
output_dir = os.getcwd() + '/results'

ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".xlsx", log_variables=list())

# these variables are saved to the harddisk after / during the time series loop

ow.log_variable('res_load', 'p_mw')
ow.log_variable('res_gen', 'p_mw')
ow.log_variable('res_bus', 'vm_pu')
ow.log_variable('res_line', 'loading_percent')
ow.log_variable('res_line', 'i_ka')


# Step 5 - Run Timeseries Powerflow

In [ ]:
run_timeseries(net, time_steps, run=pp.runpp)

You can verify in your jupyter notebook folder that the results have been exported, lets read the dataframes.

In [ ]:
print('######### Results for voltage in p.u. ########')
vm_pu_file = output_dir + '/res_bus/vm_pu.xlsx'
vm_pu = pd.read_excel(vm_pu_file, index_col=0)
print(vm_pu)

print('#######  Results for line loading in % ########')
ll_file = output_dir + '/res_line/loading_percent.xlsx'
line_loading = pd.read_excel(ll_file, index_col=0)
print(line_loading)

print('#######  Results for load power in MW ########')
load_file = output_dir + '/res_load/p_mw.xlsx'
load = pd.read_excel(load_file, index_col=0)
print(load)

print('#######  Results for generation power in MW ########')
gen_file = output_dir + '/res_gen/p_mw.xlsx'
gen = pd.read_excel(gen_file, index_col=0)
print(gen)

# Step 6 - Plotting

We can use the plotly express library to easily plot the timeseries powerflow results

In [ ]:
# plotting voltage and line results

fig = px.line(vm_pu, y=vm_pu.columns, labels={"value": "count", "variable": "Bus ID"})
fig.update_xaxes(title='Hour', fixedrange=True)
fig.update_yaxes(title='Voltage [p.u.]')
fig.update_traces(mode="markers+lines", hovertemplate='Hour: %{x} <br>Voltage: %{y}')
fig.show()

fig = px.line(line_loading, y=line_loading.columns, labels={"value": "count", "variable": "Line ID"})
fig.update_xaxes(title='Hour', fixedrange=True)
fig.update_yaxes(title='Loading [%]')
fig.update_traces(mode="markers+lines", hovertemplate='Hour: %{x} <br>Loading: %{y}')
fig.show()

In [ ]:
# plotting load and generation results

fig = px.line(load*1000, y=load.columns, labels={"value": "count", "variable": "Load ID"})
fig.update_xaxes(title='Hour', fixedrange=True)
fig.update_yaxes(title='Power [kW]')
fig.update_traces(mode="markers+lines", hovertemplate='Hour: %{x} <br>Power: %{y}')
fig.show()


fig = px.line(gen*1000, y=gen.columns, labels={"value": "count", "variable": "Gen ID"})
fig.update_xaxes(title='Hour', fixedrange=True)
fig.update_yaxes(title='Power [kW]')
fig.update_traces(mode="markers+lines", hovertemplate='Hour: %{x} <br>Power: %{y}')
fig.show()

## Try it yourself! What if we have to create a different profile for each load and generator?

In your jupyter notebook folder you will find a file named '24h_profiles.xslx'. This is an excel table with different profiles for a 24h period.

In [ ]:
net = ?
profiles_df = pd.read_excel('24h_profiles.xlsx', index_col=0)
print(profiles_df)

In this table, there are two load profiles and 1 generator profile. Apply these to the elements in the network.

For this, we need to create a loop to iterate over the loads dataframe

In [ ]:
# lets duplicate the two load profiles since we have 4 loads
profiles_df['load2_p'] = profiles_df['load0_p']
profiles_df['load3_p'] = profiles_df['load1_p']

# iterate over the elements to create a specific profile for each load
for loads in range(4):
    profiles_df.loc[:, 'load'+str(loads)+'_p'] = ?

# iterate over the elements to create a specific profile for each generator
for gen in range(1):
    ?

In [ ]:
profiles_df

Lets now create a new datasource with the 24h profiles

In [ ]:
ds_new = ?

Next step is to create a different controller for each element in the network. Similarly we can use the for loop for that.

In [ ]:
net.gen

In [ ]:
for i in net.gen.index:
    
    ConstControl(net, element='gen', variable='p_mw', element_index=?,
                 data_source=?, profile_name=?, scale_factor=1)

    
for j in net.load.index:
    
    ?

Finally, update the time_steps variable and setup the OutputWriter.

In [ ]:
time_steps = range(0, profiles_df.shape[0])
output_dir = os.getcwd() + '/results'

ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".xlsx", log_variables=list())

ow.log_variable('res_load', 'p_mw')
ow.log_variable('res_gen', 'p_mw')
ow.log_variable('res_bus', 'vm_pu')
ow.log_variable('res_line', 'loading_percent')
ow.log_variable('res_line', 'i_ka')

Now run the timeseries powerflow and check the results!

In [ ]:
?

Use the previous plotting script to check the results of the timeseries powerflow.

# Exercise 5 and Report Information

Using QGIS, calculate the distance between the isolated points and the actual transmission grid and create new lines and generators to solve the grid congestions (all lines must have less than 80% of loading) and voltage deviations (all buses must have a voltage between 0.9 and 1.1 p.u.).

You are free to choose any solution you may find useful, as long as you add at least 1 line and 1 new generator.

In your report, you must show the timesseries power flow results (res_bus and res_line tables and line plot of all of the hours as well as the map plottings of the heaviest hour) **before** and **after** your intervention. Also add a brief text explaining the changes you made in the network and why.

Grading: 

1. Exercises 1-3: 3 Points total (1 each)
2. Exercise 4: 3 Points
3. Exercise 5: 4 Points

Total 10 points

#### DEADLINE: Upload your repot in pdf format with all 5 exercies in the Atenea portal until 03.11 (Morning Group) / 24.11 (Night Group)